In [160]:
import pandas as pd
import numpy as np 
import re, json, os , logging , random, html
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender
from lib.issue_provider import issue_provider
from lib.questionnaire_parser import questionnaire_parser
from lib.questionnaire_picklist_parser import questionnaire_picklist_parser
from lib.script_generator import script_generator 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from bs4 import BeautifulSoup  
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore')


In [ ]:
def write_excel(filename,sheetname,dataframe):
    with pd.ExcelWriter(filename, engine='openpyxl', mode='a') as writer: 
        workBook = writer.book
        try:
            workBook.remove(workBook[sheetname])
        except:
            print("Worksheet does not exist")
        finally:
            dataframe.to_excel(writer, sheet_name=sheetname,index=False)
            writer.save()

In [ ]:
df=pd.read_excel(r'C:\Users\Tim\Downloads\Bill-D-Robertson_TIC_CloudServices.xlsx',  sheet_name='Sheet1', header=0 ) 
CSP=list(pd.read_excel(r'C:\Users\Tim\Downloads\Bill-D-Robertson_TIC_CloudServices.xlsx',  sheet_name='CSP').iloc[:, 0])
ServiceModelType=list(pd.read_excel(r'C:\Users\Tim\Downloads\Bill-D-Robertson_TIC_CloudServices.xlsx',  sheet_name='ServiceModelType' ).iloc[:, 0])
Remediated=list(pd.read_excel(r'C:\Users\Tim\Downloads\Bill-D-Robertson_TIC_CloudServices.xlsx',  sheet_name='Remediated').iloc[:, 0])
for k in df.columns:
    print(f"d['{k}'] = ''")
lod=[]  
for i in range(1,10):
    d={}
    d['Cloud Service Provider'] = CSP[ random.randint(0, len(CSP)) -1 ]
    d['FedRAMP Package ID'] = f'ID{i}'
    d['Service Type'] = ServiceModelType[ random.randint(0, len(ServiceModelType)) -1 ]
    d['Service Model Type'] = ServiceModelType[ random.randint(0, len(ServiceModelType)) -1 ]
    d['ATO Letter with FedRAMP PMO (Yes or No)'] = 'Yes'
    d['Comments'] = f'Comments{i}'
    lod.append(d)
lod
df=pd.DataFrame(lod) 
write_excel(r'C:\Users\Tim\Downloads\Bill-D-Robertson_TIC_CloudServices.xlsx','Sheet1',df)
df=pd.read_excel(r'C:\Users\Tim\Downloads\Bill-D-Robertson_TIC_CloudServices.xlsx',  sheet_name='Sheet1', header=0 ) 
df


In [ ]:
ip = issue_provider(ctx)  
df=ip.parse_metrics('9311')
df

In [ ]:
ip.driver.quit()

In [ ]:

d={'a':'L', 'b':'M', 'c':'H'}
for i in range(1,11):
    for k in d.keys(): 
        print(f"""
            IF @ANS_7_{i}_{k} > @SUM_111_222_{d[k]}
            BEGIN
                INSERT INTO @ErrorTable  
                SELECT  PK_Question, PK_FormPage, page_sort_pos, questgroup_sort_pos, quest_sort_pos
                , ('The number provided for ' + identifier_text + ' should not exceed the value of 1.1.1 {d[k]} + 1.1.2 {d[k]} : '  + QuestionText) Error, identifier_text
                FROM vw_OrgSubQuestions WHERE  identifier_text='7.{i}.{k}' AND PK_OrgSubmission = @PK_OrgSubmission  
            END""")

In [ ]:
with open(ctx.get_tempalte_dir()+'sum.sql', 'r') as f:
    s=f.read() 
df=sql_todf("""
SELECT DISTINCT IdText AS [{idt}], REPLACE(IdText, '.','_') [{var}],SectionNum, SortPos FROM vwQuestions  
WHERE PK_FORM = '2023-Q1-CIO' AND IdText LIKE '4.1._._' ;
""", connstr) 

df['{var}'] = df['{var}'].apply(lambda s : '@SUM_'+re.sub('_[a-z]$','',s))
df['{idt}'] = df['{idt}'].apply(lambda s : f'\'{s}\'') 
df = df.groupby('{var}', as_index=False).agg({'{idt}':list})
df

In [ ]:

df['sql']=[s.replace('{idt}',','.join(x)).replace('{var}',y) for x,y in zip(df['{idt}'], df['{var}'])]
with open(ctx.get_dest()+'script.sql', 'w') as f:
    f.write( '\n'.join(list(df['sql'])) ) 
df 

In [ ]:
with open(ctx.get_tempalte_dir()+'frmval.sql', 'r') as f:
    s=f.read() 
df=sql_todf("""
SELECT DISTINCT IdText AS [{idt}], REPLACE(IdText, '.','_') [{var}], SortPos  FROM vwQuestions  
WHERE PK_FORM = '2023-Q1-CIO'  AND IdText LIKE '7.%' AND Q_TypeID=2 
ORDER BY SortPos;
 
""", connstr) 
df['sql']=[s.replace('{idt}',x).replace('{var}',y) for x,y in zip(df['{idt}'], df['{var}'])]
with open(ctx.get_dest()+'script.sql', 'w') as f:
    f.write( '\n'.join(list(df['sql'])) ) 

In [ ]:


df['sql']=s
df['sql']=[x.replace('{idt}',y).replace('{var}',z) for x,y,z in zip(df['sql'], df['{idt}'], df['{var}'])]
 
with open(ctx.get_dest()+'script.sql', 'w') as f:
    f.write( '\n'.join(list(df['sql'])) ) 
df

In [ ]:
df,code=generate_code_from_db(ctx,4019) 
#generate_section(4018, code)
df[:10]
#sql=df_to_sql(df,ctx,template_name='fsma_QuestionsInsert.sql')

In [ ]:
def generate_section(section, code=''): 
    df=sql_todf("SELECT DISTINCT PK_QuestionGroup, ASPX FROM vwQuestions  WHERE PK_FORM = '2023-Q1-CIO'", connstr).set_index('PK_QuestionGroup' )
    d=df.T.to_dict(orient='records')[0] 
    print(d)
    path=f'C:\\dev\\CyberScope\\CyberScopeBranch\\CSwebdev\\code\\CyberScope\\{d[section]}'
    with open(path, 'r', encoding='utf-16') as f:
        s=f.read() 
    pname = d[section][d[section].rfind('/')+1:]
    if '<!--questions-->' in s:
        q=re.findall('<!--questions-->.+<!--\/questions-->',s,re.DOTALL)[0]
        s=s.replace(q,f'<!--questions-->\n{code}\n<!--/questions-->')
        with open(f"{ctx.get_dest()}{pname}", 'w', encoding='utf-16') as f:
            f.write(s) 



In [ ]:

def parse_grid(tnum, pk, QCol='Question', PivotCols=4): 
    driver.get(f"https://dayman.cyber-balance.com/jira/si/jira.issueviews:issue-html/CS-{tnum}/CS-{tnum}.html")   
    ldf = pd.read_html(driver.page_source, match="\d\.\d\.\d", header=1)
    df=ldf[0].iloc[:,:].rename(columns={QCol: "QuestionText"})
 
    df['identifier_text'] = df['QuestionText'].apply(lambda s: re.findall('^[\d\.\w]{1,7}',s)[0])
    stypes = {k: chr(v+97) for v, k in enumerate(list(df.columns[1:4]))} 
    print(stypes)
    df=df.loc[:,['QuestionText','identifier_text']] 
    df['CTRLCODE']='LABEL'
    lod=df.to_dict(orient='records')
    l=[]
    for i,r in enumerate(lod):
        l.append(lod[i])
        for k in stypes.keys():
            id=lod[i]['identifier_text']+'.'+stypes[k]
            l.append({
                'identifier_text':id
                , 'QuestionText': id+' '+k
                , 'CTRLCODE':'CNT'
                })
    df=pd.DataFrame(l)
    df['{PK_Question}'] = range(pk,len(df)+pk)
    df['sortpos'] = range(1,len(df)+1) 
    df['FK_QuestionType'] = df['CTRLCODE'].apply(lambda s: 2 if s=='CNT' else 18)
    df['QuestionText']=df['QuestionText'].apply(lambda s: re.sub('^[\d\.\w]{1,7}','',s, count=1).strip())
    df['{QuestionText}']=df['QuestionText']
    df['{idt}'] = df['identifier_text'].str.replace('.','_')
    with open (ctx.get_tempalte_dir()+'fsma_QuestionsInsert.sql', 'r') as f:
        df['sql']=f.read() 
    for i,r in df.iterrows():
        for c in df.columns: 
            df.loc[i,'sql']= re.sub(str(c),str(df.loc[i,c]),df.loc[i,'sql'])  
    sql='\n,'.join(df['sql'])
    with open(f'{ctx.get_dest()}script.sql', 'w') as f:
        f.write(sql)

    gen=script_generator(ctx)
    code=gen.generate(df)
    with open(f'{ctx.get_dest()}script.aspx', 'w') as f:
        f.write(code)
    
    return df,sql,code

# df,sql,code=parse_grid('9295',37400, QCol='Type of Test')
# print(sql)

In [ ]:
d, lod={'a':'1.1.1', 'b':'1.1.2'},[]
 
for i in range(1,7):
    for k in d.keys(): 
        print(f"""
            IF @ANS_7_1_{i}_{k} > @SUM_{d[k].replace('.','')}
            BEGIN
                INSERT INTO @ErrorTable  
                SELECT  PK_Question, PK_FormPage, page_sort_pos, questgroup_sort_pos, quest_sort_pos
                , ('The number provided for ' + identifier_text + ' should not exceed the value of {d[k]} : '  + QuestionText) Error, identifier_text
                FROM vw_OrgSubQuestions WHERE  identifier_text='7.1.{i}.{k}' AND PK_OrgSubmission = @PK_OrgSubmission  
            END""")

In [109]:
re.findall('\d{1,2}\.\d{1,2}\.?\d{0,2}\.?[\.\d\w]?', '2.1. asdfasdfasdf')[0]

'2.1.'

In [198]:
ctrl=''
with open(f'{ctx.get_tempalte_dir()}TXT_OPT.aspx', 'r') as f:
    ctrl=f.read() 
 
def cell_provider(s, i): 
    if type(s) != str : return s
    m=re.findall('\d{1,2}\.\d{1,2}\.?\d{0,2}\.?[\.\d\w]?', s)
    d={0:'a.i',1:'a.ii',2:'b.i',3:'b.ii',4:'c.i',5:'c.ii' } 
    idt=f'{m[0]}.{d[i]}'
    c=ctrl.replace('{idt}',idt.replace('.','_'))
    return f'{c}'  if len(m) > 0 else ''

def lbl_provider(s): 
    if type(s) != str : return s
    lbl='<uc:CBlabel ID="CBlabel_{idt}_i" PK_key="{idt}" runat="server" LabelType="Question" />'   
    m=re.findall('\d{1,2}\.\d{1,2}\.?\d{0,2}\.?[\.\d\w]?', s) 
    idt=f'{m[0]}'
    c=lbl.replace('{idt}',idt.replace('.','_'))
    return c
def get_ids():
    dff=sql_todf("SELECT replace(idtext, '.','_') id, pk_question  FROM vwQuestions WHERE PK_QuestionGroup=4015", ctx.config['connstr'])
    ids = {k:v for k, v in zip(dff['id'], dff['pk_question'] )  }
    return ids
ids=get_ids() 
ids
 

{'2_1': 37000,
 '2_1_a': 37001,
 '2_1_b': 37002,
 '2_1_c': 37003,
 '2_2': 37004,
 '2_2_a': 37005,
 '2_2_b': 37006,
 '2_2_c': 37007,
 '2_3': 37008,
 '2_3_a': 37009,
 '2_3_b': 37010,
 '2_3_c': 37011,
 '2_3_1': 37012,
 '2_3_1_a': 37013,
 '2_3_1_b': 37014,
 '2_3_1_c': 37015,
 '2_3_2': 37016,
 '2_3_2_a': 37017,
 '2_3_2_b': 37018,
 '2_3_2_c': 37019,
 '2_4': 37020,
 '2_4_a': 37021,
 '2_4_b': 37022,
 '2_4_c': 37023,
 '2_5': 37024,
 '2_5_a': 37025,
 '2_5_b': 37026,
 '2_5_c': 37027,
 '2_5_1': 37028,
 '2_5_1_a': 37029,
 '2_5_1_b': 37030,
 '2_5_1_c': 37031,
 '2_6': 37032,
 '2_6_a': 37033,
 '2_6_b': 37034,
 '2_6_c': 37035,
 '2_7': 37036,
 '2_7_a': 37037,
 '2_7_b': 37038,
 '2_7_c': 37039,
 '2_8': 37040,
 '2_8_a': 37041,
 '2_8_b': 37042,
 '2_8_c': 37043,
 '2_9': 37044,
 '2_9_a': 37045,
 '2_9_b': 37046,
 '2_9_c': 37047,
 '2_9_1': 37048,
 '2_9_1_a': 37049,
 '2_9_1_b': 37050,
 '2_9_1_c': 37051,
 '2_9_2': 37052,
 '2_9_2_a': 37053,
 '2_9_2_b': 37054,
 '2_9_2_c': 37055,
 '2_9_3': 37056,
 '2_9_3_a': 37057,


In [ ]:

ids

In [196]:
df=pd.read_excel(r'C:\Users\Tim\Documents\CIO\CIO_SECTION_2.xlsx',  sheet_name='All', header=0 ) # Enterprise Identities  # Public Identities 
#df.fillna('', inplace=True)
df.drop([0], axis=0, inplace=True)
df.rename(columns={'Unnamed: 2':'.','Unnamed: 4':'..','Unnamed: 6':'...'}, inplace=True)
for i,c in enumerate(df.columns[1:]): 
    df[c] = df['Question'].apply(lambda s:  cell_provider(s,i) )
df['Question']=df['Question'].apply(lbl_provider )
df.to_html(f'{ctx.get_dest()}parsed.html',index=False)
df[:5]



,Question,Number of FISMA High Systems,.,Number of FISMA Moderate Systems,..,Number of FISMA Low Systems,...
1,"<uc:CBlabel ID=""CBlabel_2_1_i"" PK_key=""2_1"" ru...","<uc:CBnumeric2 ID=""CBNumeric_2_1_a_i"" PK_Quest...","<uc:CBnumeric2 ID=""CBNumeric_2_1_a_ii"" PK_Ques...","<uc:CBnumeric2 ID=""CBNumeric_2_1_b_i"" PK_Quest...","<uc:CBnumeric2 ID=""CBNumeric_2_1_b_ii"" PK_Ques...","<uc:CBnumeric2 ID=""CBNumeric_2_1_c_i"" PK_Quest...","<uc:CBnumeric2 ID=""CBNumeric_2_1_c_ii"" PK_Ques..."
2,"<uc:CBlabel ID=""CBlabel_2_1_1_i"" PK_key=""2_1_1...","<uc:CBnumeric2 ID=""CBNumeric_2_1_1_a_i"" PK_Que...","<uc:CBnumeric2 ID=""CBNumeric_2_1_1_a_ii"" PK_Qu...","<uc:CBnumeric2 ID=""CBNumeric_2_1_1_b_i"" PK_Que...","<uc:CBnumeric2 ID=""CBNumeric_2_1_1_b_ii"" PK_Qu...","<uc:CBnumeric2 ID=""CBNumeric_2_1_1_c_i"" PK_Que...","<uc:CBnumeric2 ID=""CBNumeric_2_1_1_c_ii"" PK_Qu..."
3,"<uc:CBlabel ID=""CBlabel_2_2_i"" PK_key=""2_2"" ru...","<uc:CBnumeric2 ID=""CBNumeric_2_2_a_i"" PK_Quest...","<uc:CBnumeric2 ID=""CBNumeric_2_2_a_ii"" PK_Ques...","<uc:CBnumeric2 ID=""CBNumeric_2_2_b_i"" PK_Quest...","<uc:CBnumeric2 ID=""CBNumeric_2_2_b_ii"" PK_Ques...","<uc:CBnumeric2 ID=""CBNumeric_2_2_c_i"" PK_Quest...","<uc:CBnumeric2 ID=""CBNumeric_2_2_c_ii"" PK_Ques..."
4,"<uc:CBlabel ID=""CBlabel_2_3_i"" PK_key=""2_3"" ru...","<uc:CBnumeric2 ID=""CBNumeric_2_3_a_i"" PK_Quest...","<uc:CBnumeric2 ID=""CBNumeric_2_3_a_ii"" PK_Ques...","<uc:CBnumeric2 ID=""CBNumeric_2_3_b_i"" PK_Quest...","<uc:CBnumeric2 ID=""CBNumeric_2_3_b_ii"" PK_Ques...","<uc:CBnumeric2 ID=""CBNumeric_2_3_c_i"" PK_Quest...","<uc:CBnumeric2 ID=""CBNumeric_2_3_c_ii"" PK_Ques..."
5,"<uc:CBlabel ID=""CBlabel_2_3_1_i"" PK_key=""2_3_1...","<uc:CBnumeric2 ID=""CBNumeric_2_3_1_a_i"" PK_Que...","<uc:CBnumeric2 ID=""CBNumeric_2_3_1_a_ii"" PK_Qu...","<uc:CBnumeric2 ID=""CBNumeric_2_3_1_b_i"" PK_Que...","<uc:CBnumeric2 ID=""CBNumeric_2_3_1_b_ii"" PK_Qu...","<uc:CBnumeric2 ID=""CBNumeric_2_3_1_c_i"" PK_Que...","<uc:CBnumeric2 ID=""CBNumeric_2_3_1_c_ii"" PK_Qu..."


In [202]:

s=''
with open(f'{ctx.get_dest()}parsed.html', 'r') as f:
    s=f.read()
s=re.sub('<th>\.{1,3}</th>','',s)
s=re.sub('<th>Number','<th colspan="2">Number',s)
for k in ids:
    s=s.replace(f'PK_key="{k}"',f'PK_key="{str(ids[k])}"' )
    s=s.replace(f'PK_Question="{k}"',f'PK_Question="{str(ids[k])}"' )
with open(f'{ctx.get_dest()}script.html', 'w') as f:
    f.write(html.unescape(s))
    


In [ ]:
d, lod={'a':'L', 'b':'M', 'c':'H'},[]
 
for i in range(1,12):
    for k in d.keys(): 
        lod.append({
            'SectionText':'S2'
            ,'MetricXPath':f'//*[contains(@class, \'qid_2_{i}_{k}\')]'
            ,'ErrorAttemptExpression':f'( 1.1.1{d[k]} + 1.1.2{d[k]} ) + 1 '
            ,'ExpectedError':f'2.{i}.{k} should not exceed'
        }) 
df=pd.DataFrame(lod)
df.to_csv(r'D:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope.Tests\Selenium\TestData\Q1.csv', index=False)
 

In [61]:
df=pd.read_csv(r'C:\Users\Tim\Downloads\22-01_Help_Desk_Analysis_12022022 Submission.csv') 

df=df.applymap(lambda s: str(s).replace(',',''))
lod=df.to_dict(orient='records')
s=''
for d in lod:
    s=s+f"""
UPDATE CVEUnremediated 
SET CDMCountSubmitted={d['cdm_findings']},SubmissionCount={d['cdm_findings']}
WHERE PK_OrgSubmission = (
    SELECT MAX(PK_OrgSubmission) FROM vwOrgSubToComponent
    WHERE Acronym='{d['acronym']}'  
    AND PK_ReportingCycle=108
    AND OrgSub_Description = 'ACTIVE'
) 
AND PK_CISA_CVE='{d['cve_id']}'
    """
with open(f'{ctx.get_dest()}script.sql', 'w') as f:
    f.write(s)
